In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re


seed_regex = "\([0-9]+\)"
team_name_regex = "[A-Za-z-& ]+"
team_score_regex = "[0-9]*"

# download page data
def getPageData(year):
    url = 'http://www.databasesports.com/ncaab/tourney.htm?yr='+str(year)
    page2010 = urlopen(url)
    return BeautifulSoup(page2010, 'html.parser')

def genBrackData():
    bracket = []
    #1985 is the first year of 64 teams
    years = range(2010, 1938, -1)
    for year in years: 
        soup = getPageData(year)
        all_TRs = soup.find_all("tr")
        team_count = 0
        for iterTR in all_TRs:
            # grab class attr        
            if 'class' in iterTR.attrs:
                row_class = iterTR["class"]
                if 'region' in row_class:
                    # get region name
                    currRegion = iterTR.td.get_text()
                if 'tourney' in row_class:            
                    seed_team_score = re.findall(seed_regex + team_name_regex + team_score_regex, iterTR.td.get_text())            
                    if len(seed_team_score) > 0:
                        seed1_re = re.findall(seed_regex, seed_team_score[0])
                        team1_re = re.findall(team_name_regex, seed_team_score[0])
                        if len(seed1_re) > 0:                        
                            seed1 = seed1_re[0].replace("(","").replace(")","")
                        if len(team1_re) > 0:
                            team1 = team1_re[0].strip()
                        seed2_re = re.findall(seed_regex, seed_team_score[1])
                        team2_re = re.findall(team_name_regex, seed_team_score[1])
                        if len(seed2_re) > 0:
                            seed2 = seed2_re[0].replace("(","").replace(")","")
                        if len(team2_re) > 0:    
                            team2 = team2_re[0].strip()            
                    bracketRow = [year, currRegion, seed1, team1, currLoc, seed2, team2]
                    bracket.append(bracketRow)           
                    team_count += 1
                    if team_count >= 32:
                        break
            else:
                locs = iterTR.select("td.location")
                if locs:
                    currLoc = locs[0].a.get_text()
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Seed', 'Team Name','Location', 'Oppenent Seed', 'Opponent'])    
 
df = genBrackData()


In [7]:
df.to_csv('ncaa.csv')